In [570]:
###########################################
###########################################
############## Data Munging ###############
###########################################
###########################################


import re 
import pandas as pd 
import numpy as np 
from collections import Counter
# loading packages/aliasing

In [571]:
Reviews = pd.read_csv("Charlie.csv", encoding = "latin1")
# Read in the dataframe, normal coding does not work so I used "latin1"

In [572]:
Reviews.Score = Reviews.Score.map(str)
# Despite wanting Score in integer form, temporarily converting to string in order to 
# apply strip to the entire Data Frame
Reviews = Reviews.apply(lambda x: x.str.strip())
# Stripping white-space types from the data frame


In [573]:
# There are a few shows that I am not using, so I am going to remove those now.
set(Reviews["Show_Name"])
Reviews = Reviews.loc[(Reviews["Show_Name"] != "Indecent") & (Reviews["Show_Name"] != "Marvin's Room") \
               & (Reviews["Show_Name"] != "1984 (Broadway)")]

In [574]:
Reviews = Reviews[["Show_Name", "Review_Date", "Score", "Level", "Title", "User", "Name_Crit"]]
# Selecting only the columns I want 

In [575]:
Reviews.Level = Reviews.Level.str.findall("positive|mixed|negative")
# Extract out only "positive", "negative", and "mixed" from the string. 
Reviews.Level = pd.Series([i[0] for i in Reviews.Level])
# convert back to a series of strings

In [576]:
Reviews = Reviews.loc[~Reviews.Title.isnull()]
# removing the rows without a Title review. (approx 19)

In [577]:
Reviews.columns = ["Show_Name", "Review_Date", "Score", "Review_Strength", "Description", "Reviewer", "Critic_Name"]
# Changing some of the Column Names to be a bit more descriptive. 
Reviews.columns

Index(['Show_Name', 'Review_Date', 'Score', 'Review_Strength', 'Description',
       'Reviewer', 'Critic_Name'],
      dtype='object')

In [578]:
np.sum(Reviews.isnull(), axis=0)
# There are a ton of missings in Critic_Name but I am not really using this column- It is only 
# there to give some reference for the review. 

# As for review_strength, I was able to determine the criteria from the site for positive, negative, and mixed. 
# I will create these values for the missing ones in the next cell. 

Reviews.Score = Reviews.Score.map(int)

In [579]:
y = []
for i in Reviews["Score"]:
    if i >= 70:
        y.append("positive")
    if i <= 69 and i >= 50:
        y.append("mixed")
    if i <= 49:
        y.append("negative")
# the above loop is creating a Review_Strength value based upon the proper score. Below I proceed to drop the old 
# column and add in y as the new column. I know this is not the best way to do this but I continued to get errors 
# when attempting to reassign the values using slices so I decided to do it like this instead. 

Reviews.drop("Review_Strength", axis=1)
# drop the old column 
Reviews.reset_index(drop = True)
# reset the index in order to properly add in the new column
Reviews["Review_Strength"] = y
# make the edited Review_Strength

In [580]:
pd.options.mode.chained_assignment = None
# supposed to allow me to slice like in R
Reviews.Description.loc[Reviews.Description == ","] = "No Comment"
# changed the Critic Reviews from "," to "No Comment" for clarity

In [581]:
np.sum(Reviews.isnull(), axis=0)

Show_Name              0
Review_Date            0
Score                  0
Review_Strength        0
Description            0
Reviewer               9
Critic_Name        90906
dtype: int64

In [582]:
Reviews.head()

,Show_Name,Review_Date,Score,Review_Strength,Description,Reviewer,Critic_Name
0,Hamilton,"August 6th, 2015",100,positive,No Comment,Ben Brantley,The New York Times
1,Hamilton,"August 6th, 2015",100,positive,No Comment,Zachary Stewart,Theatermania
2,Hamilton,"August 6th, 2015",80,positive,No Comment,Elisabeth Vincentelli,New York Post
3,Hamilton,"August 7th, 2015",90,positive,No Comment,Michael Dale,BroadwayWorld
4,Hamilton,"August 6th, 2015",100,positive,No Comment,Marilyn Stasio,Variety


In [583]:
BPrice = pd.read_csv("BPrice_Final.csv")

In [584]:
BPrice.head()

,Show_Name,Show_Duration,Theater,Capacity,Performance_Date,Performance_Month,Performance_Day,Performance_Time,Minimum_Price,Maximum_Price
0,Aladdin,"2hrs, 30mins",New Amsterdam Theatre,1801,2017-08-01,8,3,19:00:00,57.5,227.5
1,Aladdin,"2hrs, 30mins",New Amsterdam Theatre,1801,2017-08-02,8,4,13:00:00,57.5,215.5
2,Aladdin,"2hrs, 30mins",New Amsterdam Theatre,1801,2017-08-02,8,4,19:00:00,57.5,215.5
3,Aladdin,"2hrs, 30mins",New Amsterdam Theatre,1801,2017-08-03,8,5,19:00:00,57.5,215.5
4,Aladdin,"2hrs, 30mins",New Amsterdam Theatre,1801,2017-08-04,8,6,20:00:00,57.5,215.5


In [585]:
# I will join on Show_Name, but I want to make sure all the show names are identical. 

print(set(Reviews.Show_Name))
set(BPrice.Show_Name)

# Show_Names that need to change: A Bronx Tale: The Musical, On Your Feet, Charlie and the Chocolate Factory, 
# School of Rock - The Musical, Beautiful- The Carol King Musical, Natasha, Pierre & the Great Comet of 1812, 
# A Doll's House Part 2, 

{'Hamilton', 'Groundhog Day The Musical', 'Wicked', 'The Book of Mormon', 'Kinky Boots', 'On Your Feet!', 'Cats', 'Anastasia', 'Charlie and The Chocolate Factory', 'War Paint', 'The Play That Goes Wrong', 'School of Rock - The Musical', 'Hello, Dolly!', 'Beautiful: The Carole King Musical', 'Aladdin', 'The Phantom of the Opera', 'Come From Away', 'A Bronx Tale', 'Natasha, Pierre & The Great Comet of 1812', 'The Lion King', 'Bandstand', 'Waitress', 'Miss Saigon', 'Chicago', "A Doll's House, Part 2", 'Dear Evan Hansen'}


{'A Bronx Tale: The Musical',
 "A Doll's House: Part 2",
 'Aladdin',
 'Anastasia',
 'Bandstand',
 'Beautiful - The Carole King Musical',
 'Cats',
 'Charlie and the Chocolate Factory',
 'Chicago',
 'Come From Away',
 'Dear Evan Hansen',
 'Groundhog Day',
 'Hamilton',
 'Hello, Dolly!',
 'Kinky Boots',
 'Miss Saigon',
 'Natasha, Pierre, and the Great Comet of 1812',
 'On Your Feet',
 'School of Rock',
 'The Book of Mormon',
 'The Lion King',
 'The Phantom of the Opera',
 'The Play That Goes Wrong',
 'Waitress',
 'War Paint',
 'Wicked'}

In [586]:
Reviews.Show_Name[Reviews.Show_Name == "Groundhog Day The Musical"] = "Groundhog Day"
Reviews.Show_Name[Reviews.Show_Name == "Charlie and The Chocolate Factory"] = "Charlie and the Chocolate Factory"
Reviews.Show_Name[Reviews.Show_Name == "School of Rock - The Musical"] = "School of Rock"
Reviews.Show_Name[Reviews.Show_Name == "On Your Feet!"] = "On Your Feet"
Reviews.Show_Name[Reviews.Show_Name == "Beautiful: The Carole King Musical"] = "Beautiful - The Carole King Musical"
Reviews.Show_Name[Reviews.Show_Name == "A Bronx Tale"] = "A Bronx Tale: The Musical"
Reviews.Show_Name[Reviews.Show_Name == "Natasha, Pierre & The Great Comet of 1812"] = "Natasha, Pierre, and the Great Comet of 1812"
Reviews.Show_Name[Reviews.Show_Name == "A Doll's House, Part 2"] = "A Doll's House: Part 2"
# Adjusting all the names from Reviews to be identical to the names from BPrice
# Before I join though, I want to fix up the descriptions column, I want to keep it as is but I want to be able to 
# extract information from it.

In [587]:
# I want to count the common descriptions among the movies. Without using NLP I can still have a good analysis here 
# because with my site, the Descriptions of the vast majority of users were preselected Adjectives- meaning spelling and 
# word lemmatization will not be as big of an issue here. I am going to group by show, and then find the 10 most common
# descriptions for each Show_Name. (Not Counting "No Comment" because these are just empty values). Then I will join
# this with the original table. 
Top10_Descriptions = Reviews.groupby(["Show_Name"]).agg({"Description": lambda x: Counter([j[0] for j in \
                                [str(i).split(",") for i in x] if j[0] != "No Comment"]).most_common(20)})
# This really long piece of code only looks complicated because there are a lot of steps in the lambda function. 
# It is grouping by Show_Name and aggregating like so- split the descriptions observations into separate strings
# and then for each show count the most common 10 shows, but not if the description is "No Comment" I did this all
# in one step because I am very bad with Python syntax for slicing on multi-level data frames :( -- Not worth
# wasting time for something I can do in one step. It is pretty clear what it is doing by looking at the table below. 
Top10_Descriptions

,Description
Show_Name,
A Bronx Tale: The Musical,"[(Funny, 170), (Absorbing, 110), (Great acting..."
A Doll's House: Part 2,"[(Funny, 272), (Absorbing, 172), (Clever, 110)..."
Aladdin,"[(Funny, 1844), (Delightful, 759), (Enchanting..."
Anastasia,"[(Delightful, 189), (Enchanting, 131), (Absorb..."
Bandstand,"[(Absorbing, 116), (Delightful, 104), (Ambitio..."
Beautiful - The Carole King Musical,"[(Delightful, 1192), (Absorbing, 827), (Funny,..."
Cats,"[(Clever, 490), (Delightful, 413), (Absorbing,..."
Charlie and the Chocolate Factory,"[(Funny, 216), (Clever, 66), (Disappointing, 4..."
Chicago,"[(Funny, 1120), (Clever, 451), (Absorbing, 410..."


In [588]:
Top10_Descriptions.Description["The Lion King"]
# Just checking here to make sure I can slice properly for the invidivual shows- I will probably 
# be doing this in the graphing section, so I want to be able to reference this syntax. 

[('Absorbing', 1919),
 ('Delightful', 1506),
 ('Clever', 1093),
 ('Enchanting', 1072),
 ('Ambitious', 948),
 ('Original', 766),
 ('Funny', 753),
 ('Epic', 258),
 ('Exquisite', 134),
 ('Great acting', 133),
 ('Masterful', 107),
 ('Must see', 92),
 ('Entertaining', 71),
 ('Disappointing', 67),
 ('Beautiful', 38),
 ('Overrated', 33),
 ('Intense', 32),
 ('Dizzying', 27),
 ('Banal', 19),
 ('Edgy', 14)]

In [589]:
Reviews = pd.merge(Reviews, Top10_Descriptions, left_on = "Show_Name", right_index = True)
# I have merged into Reviews the Top 10 descriptions for each show now. The last thing that I want to do before I 
# begin graphing is add in price data. For the most part I want to keep the two data frames separate , but I will 
# want to merge the Minimum Price from BPrice into Reviews for when I compare the Price with
# the score. 
Reviews

,Show_Name,Review_Date,Score,Review_Strength,Description_x,Reviewer,Critic_Name,Description_y
0,Hamilton,"August 6th, 2015",100,positive,No Comment,Ben Brantley,The New York Times,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
1,Hamilton,"August 6th, 2015",100,positive,No Comment,Zachary Stewart,Theatermania,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
2,Hamilton,"August 6th, 2015",80,positive,No Comment,Elisabeth Vincentelli,New York Post,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
3,Hamilton,"August 7th, 2015",90,positive,No Comment,Michael Dale,BroadwayWorld,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
4,Hamilton,"August 6th, 2015",100,positive,No Comment,Marilyn Stasio,Variety,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
5,Hamilton,"August 6th, 2015",100,positive,No Comment,David Cote,Time Out New York,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
6,Hamilton,"August 6th, 2015",95,positive,No Comment,Frank Scheck,The Hollywood Reporter,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
7,Hamilton,"August 6th, 2015",95,positive,No Comment,Steven Suskin,The Huffington Post,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
8,Hamilton,"August 6th, 2015",95,positive,No Comment,Jonathan Mandell,DC Theatre Scene,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."
9,Hamilton,"August 6th, 2015",95,positive,No Comment,Jesse Green,New York Magazine / Vulture,"[(Absorbing, 2459), (Original, 1238), (Ambitio..."


In [590]:
# What I actually want to merge though is one value for each show- the "minimum" Minimum_price. So I will 
# group_by and agg to get this, and then pd.merge into Reviews. 

BPrice_onlyprice = BPrice.groupby(["Show_Name"]).agg({"Minimum_Price": "min"})
Reviews = pd.merge(Reviews, BPrice_onlyprice, left_on = "Show_Name", right_index = True)
Reviews.head()

,Show_Name,Review_Date,Score,Review_Strength,Description_x,Reviewer,Critic_Name,Description_y,Minimum_Price
0,Hamilton,"August 6th, 2015",100,positive,No Comment,Ben Brantley,The New York Times,"[(Absorbing, 2459), (Original, 1238), (Ambitio...",749.0
1,Hamilton,"August 6th, 2015",100,positive,No Comment,Zachary Stewart,Theatermania,"[(Absorbing, 2459), (Original, 1238), (Ambitio...",749.0
2,Hamilton,"August 6th, 2015",80,positive,No Comment,Elisabeth Vincentelli,New York Post,"[(Absorbing, 2459), (Original, 1238), (Ambitio...",749.0
3,Hamilton,"August 7th, 2015",90,positive,No Comment,Michael Dale,BroadwayWorld,"[(Absorbing, 2459), (Original, 1238), (Ambitio...",749.0
4,Hamilton,"August 6th, 2015",100,positive,No Comment,Marilyn Stasio,Variety,"[(Absorbing, 2459), (Original, 1238), (Ambitio...",749.0


In [591]:
# Now I just want to Rename the new columns
Reviews.columns = ["Show_Name", "Review_Date", "Score", "Review_Strength", "Description", "Reviewer", \
                  "Critic_Name", "Top10_Adjectives", "Lowest_Price"]
Reviews.columns

Index(['Show_Name', 'Review_Date', 'Score', 'Review_Strength', 'Description',
       'Reviewer', 'Critic_Name', 'Top10_Adjectives', 'Lowest_Price'],
      dtype='object')

In [592]:
Reviews.to_csv("Reviews_Final.csv", index = False)
# Now I will export this to to a csv so that when I begin data visualization I will load in the cleaned data. 